In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import numpy as np

import scqubits as qubit
from scqubits import HilbertSpace, InteractionTerm, ParameterSweep, Explorer

In [2]:
qbt = qubit.Transmon(
    EJ=20.0,
    EC=0.2,
    ng=0.3,
    ncut=150,
    truncated_dim=5)


osc = qubit.Oscillator(
    omega=3.0,
    truncated_dim=5)


hilbertspace = HilbertSpace([qbt, osc])


interaction = InteractionTerm(
    g_strength=0.1,
    hilbertspace=hilbertspace,
    op1=qbt.n_operator(),
    subsys1=qbt,
    op2=osc.creation_operator() + osc.annihilation_operator(),
    subsys2=osc
)

interaction_list = [interaction]

param_name = '$\Phi_{ext}/\Phi_0$'
param_vals = np.linspace(0.25, 0.75, 100)

subsys_update_list = [qbt]


def update_hilbertspace(param_val):
    qbt.EJ = 20.0 * np.abs(np.cos(np.pi * param_val))


sweep = ParameterSweep(
    param_name=param_name,
    param_vals=param_vals,
    evals_count=15,
    hilbertspace=hilbertspace,
    subsys_update_list=subsys_update_list,
    update_hilbertspace=update_hilbertspace,
    interaction_list=interaction_list,
    generate_chi=True
)


explorer = Explorer(
    sweep=sweep,
    evals_count=10
)
explorer.interact()

Output()

In [3]:
qbt = qubit.Fluxonium(
    EJ=2.55,
    EC=0.72,
    EL=0.12,
    flux=0.0,
    cutoff=110,
    truncated_dim=7
)

osc = qubit.Oscillator(
    omega=1.0,
    truncated_dim=5
)


hilbertspace = qubit.HilbertSpace([qbt, osc])


interaction = InteractionTerm(
    g_strength=0.2,
    hilbertspace=hilbertspace,
    op1=qbt.n_operator(),
    subsys1=qbt,
    op2=osc.creation_operator() + osc.annihilation_operator(),
    subsys2=osc
)

interaction_list = [interaction]


param_name = '$\Phi_{ext}/\Phi_0$'
param_vals = np.linspace(-0.5, 0.5, 100)

subsys_update_list = [qbt]


def update_hilbertspace(param_val):
    qbt.flux = param_val


sweep = ParameterSweep(
    param_name=param_name,
    param_vals=param_vals,
    evals_count=10,
    hilbertspace=hilbertspace,
    subsys_update_list=subsys_update_list,
    update_hilbertspace=update_hilbertspace,
    interaction_list=interaction_list,
    generate_chi=True
)


explorer = Explorer(
    sweep=sweep,
    evals_count=10
)
explorer.interact()

Output()